<a href="https://colab.research.google.com/github/mengcius/pytorch-learn/blob/master/14_ResNet18%E5%AE%9E%E6%88%98CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 14_ResNet18实战CIFAR10

### renet.py

正规结构，https://github.com/kuangliu/pytorch-cifar

ResNet18：

2 loss: 0.7332910895347595
2 test acc: 0.7374

4 loss: 0.4083496630191803
4 test acc: 0.8184

10 loss: 0.2542964816093445
10 test acc: 0.8695

23 loss: 0.19039148092269897
23 test acc: 0.9016

ResNet50：

2 loss: 0.7702206373214722
2 test acc: 0.7155

6 loss: 0.5006080269813538
6 test acc: 0.8179

10 loss: 0.33192721009254456
10 test acc: 0.847


In [2]:
'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])


def test():
    net = ResNet18()
    y = net(torch.randn(1,3,32,32))
    print(y.size())

test()

torch.Size([1, 10])


### main.py

还是用自己实现的版本

In [0]:
import  torch
from    torch.utils.data import DataLoader
from    torchvision import datasets
from    torchvision import transforms
from    torch import nn, optim

# from    lenet5 import Lenet5
# from    resnet import ResNet10


def main():
    batchsz = 128

    # 加载数据集
    cifar_train = datasets.CIFAR10('cifar', True, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], # 数据归一化，R G B上的标准差
                             std=[0.229, 0.224, 0.225]) # R G B上的方标准差
    ]), download=True)
    cifar_train = DataLoader(cifar_train, batch_size=batchsz, shuffle=True)

    cifar_test = datasets.CIFAR10('cifar', False, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]), download=True)
    cifar_test = DataLoader(cifar_test, batch_size=batchsz, shuffle=True)

    x, label = iter(cifar_train).next() # 一个batch
    print('x:', x.shape, 'label:', label.shape)
    # x:torch.Size([32,3,32,32]) label:torch.Size([32])


    # 调用构建网络实例
    device = torch.device('cuda') # GPU
    # model = Lenet5().to(device)
    model = ResNet18().to(device)
    print(model) # 打印结构（初始化类里的结构，forward里打印不出来）

    criteon = nn.CrossEntropyLoss().to(device) # 定义交叉熵损失，分类问题，包含了softmax
    optimizer = optim.Adam(model.parameters(), lr=1e-3) # 定义优化器，传入网络参数


    for epoch in range(200):

        # 训练
        model.train() # 有些模型在train和eval时计算不同，要声明什么模式，如BN、Dropout
        for batchidx, (x, label) in enumerate(cifar_train): # 迭代
            x, label = x.to(device), label.to(device) # GPU，x:[b, 3, 32, 32]，label:[b]

            logits = model(x) # 调用model的forward，传进x
            # logits:[b, 10]，label:[b]
            loss = criteon(logits, label) # 计算loss，输入是logits而不是经过softmax的predect

            # backprop
            optimizer.zero_grad() # 梯度清零，否则默认累加
            loss.backward() # 计算梯度，累加到了0上面
            optimizer.step() # 沿梯度更新

        # 这打印的是此epoch中最后一个batch的loss，loss数据仅供参考，主要看TEST ACC
        print(epoch, 'loss:', loss.item()) # loss是tensor标量，要通过item转为numpy打印出来

        # 测试
        model.eval() # eval模式
        with torch.no_grad(): # 测试时不需要backprop计算梯度，不需要构建计算图，防止打乱计算图更安全
            # test
            total_correct = 0 # 全局
            total_num = 0
            for x, label in cifar_test:
                x, label = x.to(device), label.to(device)
              
                logits = model(x) # [b, 10]
                pred = logits.argmax(dim=1) # 对logits做argmax得到预测标签，[b]

                # [b] vs [b] => scalar tensor byteTensor => float，再累加，转为numpy
                total_correct += torch.eq(pred, label).float().sum().item() # 预测与真实标签逐元素比较
                total_num += x.size(0) # batch数累加

            acc = total_correct / total_num # 测试集总的准确率
            print(epoch, 'test acc:', acc)


if __name__ == '__main__':
    main()

Files already downloaded and verified
Files already downloaded and verified
x: torch.Size([128, 3, 32, 32]) label: torch.Size([128])
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d